In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/My Drive

/content/drive/My Drive


In [2]:
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

def crawler(i):
    url = 'https://www.politifact.com/factchecks/?page='+str(i)
    fail = []
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'lxml')
    except:
        print(i)
        
    results = []
    for article in soup.select('article.m-statement.m-statement--is-medium'):
        url_sub = 'https://www.politifact.com' + article.select('a')[1]['href']
        try:
            page_sub = requests.get(url_sub)
            soup_sub = BeautifulSoup(page_sub.content, 'lxml')
        except:
            fail.append(url_sub)
            continue
        
        try:
            name = soup_sub.find(class_='m-statement__name')
            name_str = name.get_text().strip()
            name_link = 'https://www.politifact.com' + name['href']
            statement_str = soup_sub.find(class_='m-statement__quote').get_text().strip()
            rating_link = soup_sub.select('picture')[1].select('img')[0]['src']

            when_where_str = soup_sub.find(class_='m-statement__desc').get_text().strip()

            title_str = soup_sub.select('h2.c-title.c-title--subline')[0].get_text().strip()
            content_html = soup_sub.find(class_='m-textblock')
            content_str = content_html.get_text().strip()
        except:
            fail.append(url_sub)
            continue
        
        try:
            factchecker = soup_sub.find(class_='m-author__content').select('a')[0]
        except:
            fail.append(url_sub)
            continue
        
        factchecker_str = factchecker.get_text().strip()
        factchecker_link = 'https://www.politifact.com' + factchecker['href']
        factcheck_date = soup_sub.find(class_='m-author__date').get_text().strip()

        results.append([url_sub, 
                        name_str, 
                        name_link, 
                        statement_str, 
                        rating_link, 
                        when_where_str, 
                        title_str, 
                        content_str, 
                        content_html, 
                        factchecker_str, 
                        factchecker_link, 
                        factcheck_date])
    return results, fail

results_all = []
fails = []
for i in range(1, 699):
    if i % 50 == 0:
        print(i)
    info = crawler(i)
    results_all.append(info[0])
    fails.append(info[1])

50
100
150
200
250
300
350
400
450
500
550
600
650


In [4]:
from functools import reduce
from operator import iconcat 
import pandas as pd

df = pd.DataFrame(reduce(iconcat, results_all), columns=['url', 
                                                         'speaker', 
                                                         'speaker_info', 
                                                         'statement', 
                                                         'rating', 
                                                         'where_when', 
                                                         'title', 
                                                         'content', 
                                                         'content_html', 
                                                         'factchecker', 
                                                         'factchecker_info', 
                                                         'factcheck_date'])

df.to_csv('politifact.csv')

In [5]:
fails = reduce(iconcat, fails)

In [10]:
df_fails = pd.DataFrame(fails, columns = ['url'])
df_fails.to_csv('fails.csv')

In [12]:
df = pd.read_csv('politifact.csv')